Exercise 1

In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Pre-procesamiento

In [33]:
# Testing

print("\n\n\n\n\nDatos:\n\n\n\n\n")
for columna in clientes_data.columns:
    print(f"\nColumna: {columna}")
    print(clientes_data[columna].unique())

# Tipos de datos
print("\nTipos de datos:\n")
for col in clientes_data.columns:
    print(f"{col}: {type(clientes_data[col][0])}")






Datos:






Columna: ID
[2174 6182 6177 ... 8080 8372 9405]

Columna: Year_Birth
[1954 1984 1985 1974 1983 1976 1952 1987 1980 1969 1989 1970 1951 1973
 1957 1959 1975 1996 1968 1977 1988 1982 1971 1967 1965 1960 1972 1979
 1964 1978 1963 1962 1953 1961 1956 1958 1981 1966 1900 1986 1893 1949
 1992 1950 1955 1946 1948 1944 1990 1991 1995 1947 1945 1940]

Columna: Education
[3 5 1 4 2]

Columna: Marital_Status
[1 4 5 3 2]

Columna: Income
[46344. 26646. 33454. 30351.    nan  7500. 59354. 17323. 41850. 33812.
 18589. 53359. 38360. 10979. 40548. 46610. 49389. 23718. 42429. 20559.
 21994. 41728. 35790. 27938. 55954. 67680. 25721. 32474. 60631. 28332.
 40246. 26326. 56046. 29760. 26304. 23559. 29440. 36138. 50388. 54178.
 42394. 23626. 30096. 47916. 51813. 47823. 34554. 24594. 49096. 38557.
 57288. 43974. 50785. 30753. 21918. 32557. 19510. 30992. 54348. 77376.
 61331. 41551.  9548. 33762. 35860. 36921. 23957. 38547. 49605. 65747.
 34176. 49967. 60199. 55375. 30523. 23228. 43482. 52332

KeyError: 0

## Importar datos, eliminar atributos, convertir formatos atributos, codificar atributos

In [40]:
# Leer los datos del archivo CSV
clientes_data = pd.read_csv('./datos_de_clientes.csv', sep='\t')

# Eliminar las columnas "Z_CostContact" y "Z_Revenue" ya que contienen el mismo valor para todas las filas, por lo que no aportan información
clientes_data = clientes_data.drop(columns=['Z_CostContact', 'Z_Revenue'])

# Convertir la columna 'Dt_Customer' a tipo datetime
clientes_data['Dt_Customer'] = pd.to_datetime(clientes_data['Dt_Customer'], format='%d-%m-%Y')

# Convertir las columnas 'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response', 'Complain' a tipo booleano
clientes_data[['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response', 'Complain']] = clientes_data[['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response', 'Complain']].astype(bool)


#Arregla los datos de estado marital, quitando los valores atipicos 
# Reemplazar los valores "YOLO" y "Absurd" por "Married"
clientes_data['Marital_Status'] = clientes_data['Marital_Status'].replace(['YOLO', 'Absurd'], 'Married')
# Reemplazar los valores "Alone" por "Single"
clientes_data['Marital_Status'] = clientes_data['Marital_Status'].replace('Alone', 'Single')

# Diccionarios de mapeo para codificación ordinal
education_mapping = {'Basic': 1, '2n Cycle': 2, 'Graduation': 3, 'Master': 4, 'PhD': 5}
marital_status_mapping = {'Single': 1, 'Widow': 2, 'Divorced': 3, 'Together': 4, 'Married': 5}

# Aplicar la codificación ordinal y reemplazar las columnas originales
clientes_data['Education'] = clientes_data['Education'].map(education_mapping)
clientes_data['Marital_Status'] = clientes_data['Marital_Status'].map(marital_status_mapping)


#Revisar si hay espacio vacios
# Iterar sobre las filas del DataFrame
for index, row in clientes_data.iterrows():
    # Verificar si hay un valor nulo en la columna 'ID'
    if pd.isnull(row['ID']):
        print("Hay un espacio vacío en el ID", index)

# Definir la función lambda para codificar las fechas
def codificar_fecha(fecha):
    if fecha.year < 2013:
        return 1
    elif fecha.year == 2013:
        return 2
    else:
        return 3

# Aplicar la función lambda a la columna 'Dt_Customer'
clientes_data['Dt_Customer'] = clientes_data['Dt_Customer'].apply(lambda x: codificar_fecha(x))


## Eliminar datos atípicos

In [41]:
index = 1
num_atipicos = 1
while num_atipicos > 0:
    print(f"\n\nIteración {index}\n")
    # Imprimir el número de filas
    print("Número de filas:", clientes_data.shape[0])
    
    # Calcular los cuartiles para cada columna
    Q1_income = clientes_data['Income'].quantile(0.25)
    Q3_income = clientes_data['Income'].quantile(0.75)
    IQR_income = Q3_income - Q1_income

    Q1_recency = clientes_data['Recency'].quantile(0.25)
    Q3_recency = clientes_data['Recency'].quantile(0.75)
    IQR_recency = Q3_recency - Q1_recency

    # Calcula Q1, Q3 e IQR para las columnas relacionadas con el gasto en productos
    columns_mnt = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
    Q1_mnt = clientes_data[columns_mnt].quantile(0.25)
    Q3_mnt = clientes_data[columns_mnt].quantile(0.75)
    IQR_mnt = Q3_mnt - Q1_mnt

    # Calcula Q1, Q3 e IQR para las columnas relacionadas con el número de compras y visitas
    columns_purchases_visits = ['NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']
    Q1_purchases_visits = clientes_data[columns_purchases_visits].quantile(0.25)
    Q3_purchases_visits = clientes_data[columns_purchases_visits].quantile(0.75)
    IQR_purchases_visits = Q3_purchases_visits - Q1_purchases_visits

    # Identificar datos atípicos en las columnas seleccionadas
    datos_atipicos_income = (clientes_data['Income'] < (Q1_income - 1.5 * IQR_income)) | (clientes_data['Income'] > (Q3_income + 1.5 * IQR_income))
    datos_atipicos_recency = (clientes_data['Recency'] < (Q1_recency - 1.5 * IQR_recency)) | (clientes_data['Recency'] > (Q3_recency + 1.5 * IQR_recency))
    datos_atipicos_mnt = ((clientes_data[columns_mnt] < (Q1_mnt - 1.5 * IQR_mnt)) | (clientes_data[columns_mnt] > (Q3_mnt + 1.5 * IQR_mnt))).any(axis=1)
    datos_atipicos_purchases_visits = ((clientes_data[columns_purchases_visits] < (Q1_purchases_visits - 1.5 * IQR_purchases_visits)) | (clientes_data[columns_purchases_visits] > (Q3_purchases_visits + 1.5 * IQR_purchases_visits))).any(axis=1)

    # Combinar los resultados de detección de valores atípicos
    datos_atipicos = datos_atipicos_income | datos_atipicos_recency | datos_atipicos_mnt | datos_atipicos_purchases_visits
    # Imprimir el número de filas con valores atípicos
    print("Número de filas con valores atípicos:", clientes_data[datos_atipicos].shape[0])
    num_atipicos = clientes_data[datos_atipicos].shape[0]

    # Filtrar el DataFrame original para eliminar las filas con valores atípicos
    clientes_data = clientes_data[~datos_atipicos]

    # Imprimir el número de filas después de eliminar los valores atípicos
    print("Número de filas después de eliminar los valores atípicos:", clientes_data.shape[0])
    index += 1

# Muestra cuántos valores nulos hay en cada columna
print(clientes_data_sin_atipicos.isnull().sum())



Iteración 1

Número de filas: 2240
Número de filas con valores atípicos: 704
Número de filas después de eliminar los valores atípicos: 1536


Iteración 2

Número de filas: 1536
Número de filas con valores atípicos: 410
Número de filas después de eliminar los valores atípicos: 1126


Iteración 3

Número de filas: 1126
Número de filas con valores atípicos: 290
Número de filas después de eliminar los valores atípicos: 836


Iteración 4

Número de filas: 836
Número de filas con valores atípicos: 477
Número de filas después de eliminar los valores atípicos: 359


Iteración 5

Número de filas: 359
Número de filas con valores atípicos: 67
Número de filas después de eliminar los valores atípicos: 292


Iteración 6

Número de filas: 292
Número de filas con valores atípicos: 47
Número de filas después de eliminar los valores atípicos: 245


Iteración 7

Número de filas: 245
Número de filas con valores atípicos: 16
Número de filas después de eliminar los valores atípicos: 229


Iteración 8

Núm